#企業の決算情報をEDINET APIで取得する

https://non-dimension.com/get-xbrldata/

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [53]:
import glob
import json
import os
import zipfile

import requests
from bs4 import BeautifulSoup

!pip install arelle
!pip install isodate
from arelle import Cntlr, ModelManager
from arelle.ModelValue import qname

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
working_dir = "/content/drive/My Drive/05_決算書/"
os.chdir(working_dir)
data_dir = os.path.join(working_dir, "python")  # 生データ用のフォルダ名を入れる
os.makedirs(data_dir, exist_ok=True)

In [ ]:
# 書類一覧APIのエンドポイント
url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents.json"
# 書類一覧APIのリクエストパラメータ
params = {"date": "2022-04-25", "type": 2}

# 書類一覧APIの呼び出し
res = requests.get(url, params=params, verify=False)

# レスポンス(JSON)の表示
print(res.text)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{
    "metadata":
        {
            "title": "提出された書類を把握するためのAPI",
            "parameter":
                {
                    "date": "2022-04-25",
                    "type": "2"
                },
            "resultset":
                {
                    "count": 275
                },
            "processDateTime": "2022-06-19 00:01",
            "status": "200",
            "message": "OK"
        },
    "results": [
        {
            "seqNumber": 1,
            "docID": "S100NS32",
            "edinetCode": "E11764",
            "secCode": null,
            "JCN": "1010401064261",
            "filerName": "Ｔ＆Ｄアセットマネジメント株式会社",
            "fundCode": "G13450",
            "ordinanceCode": "030",
            "formCode": "10A000",
            "docTypeCode": "160",
            "periodStart": "2021-07-30",
            "periodEnd": "2022-07-29",
            "submitDateTime": "2022-04-25 09:00",
            "docDescription": "半期報告書（内国投資信託受益証券）－第3期(令和3年7月30日－令和4年7月29日)",


In [ ]:
# APIの結果をjson形式に変換
res_text = json.loads(res.text)

# res_text内のresultsの内容を取得
results = res_text["results"]

# resultsの中身(docID, docDescription, filerName)を表示
for result in results:
    print(result["docID"], result["docDescription"], result["filerName"])

S100NS32 半期報告書（内国投資信託受益証券）－第3期(令和3年7月30日－令和4年7月29日) Ｔ＆Ｄアセットマネジメント株式会社
S100NLHR 有価証券報告書（内国投資信託受益証券）－第19期(令和3年7月27日－令和4年1月25日) Ｔ＆Ｄアセットマネジメント株式会社
S100NX2W 臨時報告書（内国特定有価証券） 明治安田アセットマネジメント株式会社
S100NTT3 有価証券届出書（内国投資信託受益証券） 東京海上アセットマネジメント株式会社
S100NVMW 有価証券報告書（内国投資信託受益証券）－第6期(令和3年1月26日－令和4年1月25日) ニッセイアセットマネジメント株式会社
S100NSCX 有価証券届出書（内国投資信託受益証券） ピクテ投信投資顧問株式会社
S100NTT6 有価証券報告書（内国投資信託受益証券）－第5期(令和3年1月26日－令和4年1月25日) 東京海上アセットマネジメント株式会社
S100NPJ5 有価証券報告書（内国投資信託受益証券）－第33期(令和3年7月27日－令和4年1月25日) アムンディ・ジャパン株式会社
S100NPDJ 訂正有価証券届出書（内国投資信託受益証券） 野村アセットマネジメント株式会社
S100NVOM 有価証券報告書（内国投資信託受益証券）－第36期(令和3年7月29日－令和4年1月28日) 岡三アセットマネジメント株式会社
S100NSD0 有価証券報告書（内国投資信託受益証券）－第24期(令和3年7月27日－令和4年1月25日) ピクテ投信投資顧問株式会社
S100NTT8 有価証券届出書（内国投資信託受益証券） 東京海上アセットマネジメント株式会社
S100NX2Y 臨時報告書（内国特定有価証券） 明治安田アセットマネジメント株式会社
S100NVW3 訂正有価証券届出書（内国投資信託受益証券） 三菱ＵＦＪ国際投信株式会社
S100NWP7 有価証券報告書（内国投資信託受益証券）－第24期(令和3年7月27日－令和4年1月25日) アセットマネジメントＯｎｅ株式会社
S100NIOA 有価証券報告書（内国投資信託受益証券）－第4期(令和3年1月26日－令和4年1月25日) 日興アセットマネジメント株式会社
S100NVN3 有価証券届出書（内国投資信託受益証券） ニッセイアセッ

In [40]:
kessan = []
for result in results:
    if result["docDescription"] is not None:
        if "半期" in result["docDescription"]:
            print(result["docID"], result["docDescription"], result["filerName"])
            kessan.append(result)

S100NS32 半期報告書（内国投資信託受益証券）－第3期(令和3年7月30日－令和4年7月29日) Ｔ＆Ｄアセットマネジメント株式会社
S100NPDI 半期報告書（内国投資信託受益証券）－第8期(令和3年7月27日－令和4年7月25日) 野村アセットマネジメント株式会社
S100NSWX 半期報告書（内国投資信託受益証券）－第12期(令和3年7月27日－令和4年7月25日) 野村アセットマネジメント株式会社
S100NVW4 半期報告書（内国投資信託受益証券）－第15期(令和3年7月27日－令和4年7月22日) 三菱ＵＦＪ国際投信株式会社
S100NIOK 半期報告書（内国投資信託受益証券）－第3期(令和3年7月27日－令和4年7月25日) 日興アセットマネジメント株式会社
S100NVF7 半期報告書（内国投資信託受益証券）－第15期(令和3年7月27日－令和4年7月22日) 三菱ＵＦＪ国際投信株式会社
S100NXKP 半期報告書－第38期(令和3年8月1日－令和4年7月31日) ジャパンメディアシステム株式会社
S100NUU3 半期報告書（内国投資信託受益証券）－第24期(令和3年7月27日－令和4年7月22日) 三菱ＵＦＪ国際投信株式会社
S100NQ12 半期報告書（内国投資信託受益証券）－第22期(令和3年7月27日－令和4年7月25日) 野村アセットマネジメント株式会社
S100NVOY 半期報告書（内国投資信託受益証券）－第21期(令和3年7月27日－令和4年7月22日) 三菱ＵＦＪ国際投信株式会社
S100NVFQ 半期報告書（内国投資信託受益証券）－第24期(令和3年7月27日－令和4年7月22日) 三菱ＵＦＪ国際投信株式会社
S100NU41 半期報告書（内国投資信託受益証券）－第21期(令和3年7月27日－令和4年7月22日) 三菱ＵＦＪ国際投信株式会社
S100NU4L 半期報告書（内国投資信託受益証券）－第3期(令和3年7月27日－令和4年7月25日) ＳＯＭＰＯアセットマネジメント株式会社
S100NX4Z 半期報告書（内国投資信託受益証券）－第3期(令和3年7月27日－令和4年7月25日) ＳＯＭＰＯアセットマネジメント株式会社
S100NX21 半期報告書（内国投資信託受益証券）－第3期(令和3年7月27日－令和4年7月25

In [42]:
# 0番目のdocumentのdocIDを取得
docid = kessan[0]["docID"]
print(docid)

S100NS32


In [43]:
# 書類所得APIのエンドポイント
url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents/" + docid

os.chdir(data_dir)

# 書類取得APIのリクエストパラメータ
params = {"type": 1}

# 出力ファイル名
filename = docid + ".zip"

# 書類取得APIの呼び出し
res = requests.get(url, params=params, verify=False)

# ファイルへの出力
if res.status_code == 200:
    with open(filename, "wb") as f:
        for chunk in res.iter_content(chunk_size=1024):
            f.write(chunk)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [44]:
with zipfile.ZipFile(filename) as existing_zip:
    existing_zip.extractall(docid)

In [45]:
filepath = docid + "/XBRL/PublicDoc/"
files = glob.glob(filepath + "*.htm")  # htmファイルの取得
files = sorted(files)
target_file = files[1]
print(target_file)

S100NS32/XBRL/PublicDoc/0101033_honbun_jpsps100000-ssr-001_G13450-000_2022-01-29_01_2022-04-25_ixbrl.htm


In [46]:
with open(target_file, encoding="utf-8") as f:
    html = f.read()

In [47]:
soup = BeautifulSoup(html, "html.parser")
tag_p = soup.find_all("p")  # pタグの要素を取得

# pタグのテキストを表示
for p in tag_p:
    print(p.text)

１【ファンドの運用状況】
（１）【投資状況】
資産の種類別、地域別の投資状況
（2022年2月28日現在）
資産の種類
国/地域
時価合計（百万円）
投資比率（％）
株式
日本
56
17.56
社債券
ケイマン諸島
260
81.08
現金・預金・その他の資産（負債差引後）
日本
5
1.36
合計（純資産総額）
－
321
100.00
（注）投資比率とは、ファンドの純資産総額に対する当該資産の時価の比率をいいます。
（小数点以下第3位を四捨五入して算出しております。）
 
（２）【運用実績】
①【純資産の推移】
2022年2月末日及び同日前1年以内における各月末及び各計算期間末日の純資産の推移は次の通りです。
 
純資産総額
（分配落）
（単位:百万円）
純資産総額
（分配付）
（単位:百万円）
1口当たりの
純資産額
（分配落）
（単位:円）
1口当たりの
純資産額
（分配付）
（単位:円）
第1期 計算期間
（2020年7月29日現在）
524
524
0.9986
0.9986
2021年2月末日
397
－
1.0372
－
2021年3月末日
399
－
1.0583
－
2021年4月末日
381
－
1.0503
－
2021年5月末日
376
－
1.0507
－
2021年6月末日
363
－
1.0546
－
第2期 計算期間
（2021年7月29日現在）
344
358
1.0155
1.0555
2021年7月末日
344
－
1.0136
－
2021年8月末日
338
－
1.0147
－
2021年9月末日
334
－
1.0152
－
2021年10月末日
330
－
1.0112
－
2021年11月末日
326
－
1.0006
－
2021年12月末日
327
－
1.0041
－
2022年1月末日
324
－
0.9958
－
2022年2月末日
321
－
0.9920
－
 
②【分配の推移】
 
1口当たりの分配金（円）
第1期 計算期間（2020年7月29日）
0.0000
第2期 計算期間（2021年7月29日）
0.0400
③【収益率の推移】
 
収益率（％）
第1期 計算期間（2019年7月31日 ～ 2020年7月29日）
△0.14
第2期 計算期間（2020年7月30日 ～ 2021年7月29日）

#ArelleでXBRL解析

In [ ]:
# 初期化・インスタンス文章読み込み
ctrl = Cntlr.Cntlr()
model_manager = ModelManager.initialize(ctlr)
model_xbrl = model_manager.load()


# 文章情報を解放
model_manager.close()

In [ ]:
# プレフィックスをキーとして名前空間URIが格納されている辞書
ns = model_xbrl.prefixedNamespaces[""]

# QName文字列からQName型に変更
qn = qname(ns, name="")

# QNameをキーとしてfactが格納されている辞書
# 値はfactのset（同じQNameのfactは複数ありうる）
facts = model_xbrl.factsByQname[qn]